In [ ]:
import pandas as pd
import numpy as np  
import os
from prophet import Prophet
import matplotlib.pyplot as plt

In [ ]:
# Get the current directory
current_dir = os.getcwd()

# Navigate one folder up
parent_dir = os.path.dirname(current_dir)

# Where the files are located
data = "out/"

# Navigate down into the "data" folder
data_dir = os.path.join(parent_dir, data)

# Initialize an empty list to store DataFrames
dataframes = []

# Variable to track total rows
total_rows = 0

# Loop through all files in the "data" folder
try:
    for file_name in os.listdir(data_dir):
        
        if file_name.endswith('.csv')  and "2025" not in file_name:  # Check if the file is a CSV
            
            file_path = os.path.join(data_dir, file_name)
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            dataframes.append(df)  # Append the DataFrame to the list
            
             # Print dimensions of the current file
            print(f"File: {file_name} | Dimensions: {df.shape}")
            
            # Add the number of rows to the total count
            total_rows += df.shape[0]

    # Concatenate all DataFrames in the list by binding rows
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Print dimensions of the combined DataFrame
    print(f"Combined DataFrame Dimensions: {combined_df.shape}")

    # Verify the sum of rows matches
    if total_rows == combined_df.shape[0]:
        print("Row count verification successful! Total rows match.")
    else:
        print("Row count verification failed! Mismatch in row count.")

    print(combined_df.head())  # Display the first few rows of the combined DataFrame

except FileNotFoundError:
    print(f"Folder '{data_dir}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [4]:

combined_df.to_csv("../out/all_hotel_forecasts_05_02_2025_lagged_6m.csv", index=False)



In [89]:
combined_df = pd.read_csv("../out/all_hotel_forecasts_05_02_2025_lagged_6m.csv")

In [90]:
all_hotel_forecasts_04_02_2025_lagged_6m = combined_df

In [ ]:
all_hotel_forecasts_04_02_2025_lagged_6m

In [ ]:
# Get the current directory
current_dir = os.getcwd()

# Navigate one folder up
parent_dir = os.path.dirname(current_dir)

# Where the files are located
data = "data/other/2021_Jan2025_PricesHotelsDates"

# Navigate down into the "data" folder
data_dir = os.path.join(parent_dir, data)

# Initialize an empty list to store DataFrames
dataframes = []

# Variable to track total rows
total_rows = 0

# Loop through all files in the "data" folder
try:
    for file_name in os.listdir(data_dir):
        
        if file_name.endswith('.csv'):  # Check if the file is a CSV
            
            file_path = os.path.join(data_dir, file_name)
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            dataframes.append(df)  # Append the DataFrame to the list
            
             # Print dimensions of the current file
            print(f"File: {file_name} | Dimensions: {df.shape}")
            
            # Add the number of rows to the total count
            total_rows += df.shape[0]

    # Concatenate all DataFrames in the list by binding rows
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Print dimensions of the combined DataFrame
    print(f"Combined DataFrame Dimensions: {combined_df.shape}")

    # Verify the sum of rows matches
    if total_rows == combined_df.shape[0]:
        print("Row count verification successful! Total rows match.")
    else:
        print("Row count verification failed! Mismatch in row count.")

    print(combined_df.head())  # Display the first few rows of the combined DataFrame

except FileNotFoundError:
    print(f"Folder '{data_dir}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
combined_df['Data'] = pd.to_datetime(combined_df['Data']).dt.tz_localize(None)
combined_df.head

In [ ]:

# Filter for data in 2024
hotels_2024 = combined_df[combined_df['Data'].dt.year == 2024]['Hotel_ID'].unique()

# Filter the original combined_df to keep only hotels that appeared in 2024
combined_df = combined_df[combined_df['Hotel_ID'].isin(hotels_2024)]

# Display the filtered DataFrame
combined_df

In [94]:
combined_df = combined_df.loc[:, ['Data',  'Hotel_ID', 'DiariaMedia']]

combined_df.rename(columns={'Data': 'ds', 'Hotel_ID': 'hotel_id', 'DiariaMedia': 'y'}, inplace=True)

combined_df = combined_df.loc[:, ['hotel_id',  'ds', 'y']]

In [ ]:
combined_df = combined_df.groupby(['hotel_id', 'ds']).agg({'y': 'mean'}).reset_index()
combined_df.shape

In [ ]:
# Group by 'hotel_id' and count the number of rows for each hotel
hotel_counts = combined_df.groupby('hotel_id').size()

# Filter for hotels that have more than 92 rows
hotels_with_more_than_30_rows = hotel_counts[hotel_counts > 30].index

# Filter the original table for these hotels
filtered_combined_df = combined_df[combined_df['hotel_id'].isin(hotels_with_more_than_30_rows)]

# Display the filtered DataFrame
filtered_combined_df

In [97]:
combined_df = combined_df[combined_df['ds'] > '2024-06-30']
all_hotel_forecasts_04_02_2025_lagged_6m = all_hotel_forecasts_04_02_2025_lagged_6m[all_hotel_forecasts_04_02_2025_lagged_6m['ds'] > '2024-06-30']


In [98]:
hotel_list = all_hotel_forecasts_04_02_2025_lagged_6m['hotel'].drop_duplicates()

In [99]:
filtered_combined_df.rename(columns={'hotel_id': 'hotel'}, inplace=True)

In [ ]:
all_hotel_forecasts_04_02_2025_lagged_6m

In [100]:
filtered_combined_df["ds"] = pd.to_datetime(filtered_combined_df["ds"])
all_hotel_forecasts_04_02_2025_lagged_6m["ds"] = pd.to_datetime(all_hotel_forecasts_04_02_2025_lagged_6m["ds"])

In [ ]:
hotel_list

In [101]:
filtered_combined_df = filtered_combined_df[filtered_combined_df['hotel'].isin(hotel_list)]
all_hotel_forecasts_04_02_2025_lagged_6m = all_hotel_forecasts_04_02_2025_lagged_6m[all_hotel_forecasts_04_02_2025_lagged_6m['hotel'].isin(hotel_list)]


In [ ]:
filtered_combined_df['hotel'].nunique()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Initialize a list to store results
error_metrics = []

# Loop through each hotel
for hotel in hotel_list:
    hotel_forecast = all_hotel_forecasts_04_02_2025_lagged_6m[all_hotel_forecasts_04_02_2025_lagged_6m["hotel"] == hotel]
    historical_data = filtered_combined_df[filtered_combined_df["hotel"] == hotel]

    # Merge on the date column
    comparison = historical_data.merge(hotel_forecast, on="ds", how="inner")

    # Skip if no matching dates found
    if comparison.empty:
        print(f"Skipping hotel {hotel}: No matching dates found.")
        continue  

    # Compute error metrics
    mae = mean_absolute_error(comparison["y"], comparison["yhat"])
    rmse = np.sqrt(mean_squared_error(comparison["y"], comparison["yhat"]))
    mape = np.mean(np.abs((comparison["y"] - comparison["yhat"]) / comparison["y"])) * 100

    # Append results
    error_metrics.append({"hotel": hotel, "MAE": mae, "RMSE": rmse, "MAPE": mape})


# Convert results to a DataFrame
error_df = pd.DataFrame(error_metrics)

# Display the error metrics table
print(error_df)

# Save to CSV for further analysis
error_df.to_csv("../out/hotel_error_metrics_lagged_6m.csv", index=False)

print("Error metrics saved to 'hotel_error_metrics_lagged_6m.csv'.")


In [ ]:
error_df['MAPE'].describe()

In [104]:
# Remove rows where MAPE is infinite
error_df = error_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["MAPE"])


In [ ]:
error_df['MAPE'].describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a histogram of the "MAPE" variable
plt.figure(figsize=(8, 6))
sns.histplot(data=error_df[ (error_df['MAPE']<100) & (error_df['MAPE']>0) ], x="MAPE", bins=500, kde=False)

# Add labels and title
plt.xlabel("\n MAPE")
plt.ylabel("Frequency \n")
plt.title("Histogram of Mean Absolute Percent Error [MAPE]")

# Show the plot
plt.show()


In [ ]:
# Ensure 'ds' columns are in datetime format
filtered_combined_df["ds"] = pd.to_datetime(filtered_combined_df["ds"])
all_hotel_forecasts_04_02_2025_lagged_6m["ds"] = pd.to_datetime(all_hotel_forecasts_04_02_2025_lagged_6m["ds"])

# Merge both tables on 'hotel' and 'ds'
comparison = filtered_combined_df.merge(
    all_hotel_forecasts_04_02_2025_lagged_6m, 
    on=["hotel", "ds"], 
    how="inner"
)

In [ ]:
comparison

In [79]:
hotel_list = comparison.groupby('hotel').size().sort_values(ascending=False).head(100).index.tolist()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seaborn theme
sns.set_theme()

# Define figure size based on the number of hotels
fig, axes = plt.subplots(len(hotel_list), 1, figsize=(12, 6 * len(hotel_list)), sharex=True)

# If there's only one hotel, `axes` won't be an iterable, so wrap it in a list
if len(hotel_list) == 1:
    axes = [axes]

# Define colors for historical and forecasted data
colors = {"historical": "navy", "forecast": "firebrick"}

# Loop through each hotel and create a separate plot
for i, hotel in enumerate(hotel_list):
    # Filter data for this hotel
    hotel_forecast = comparison[comparison["hotel"] == hotel]

    # Plot historical data
    sns.lineplot(ax=axes[i], 
                 data=hotel_forecast, 
                 x="ds", y="y", label="Historical", color=colors["historical"])

    # Plot forecasted data
    sns.lineplot(ax=axes[i], 
                 data=hotel_forecast, 
                 x="ds", y="yhat", label="Forecast", color=colors["forecast"], linestyle="dashed")

    # Formatting for each subplot
    axes[i].set_title(f"Hotel {hotel} Forecast")
    axes[i].set_xlabel("Date")
    axes[i].set_ylabel("Prices [Local Currency]")
    axes[i].legend()
    axes[i].grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show plots
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seaborn theme
sns.set_theme()

hotel_list = [1969]
# Define figure size based on the number of hotels
fig, axes = plt.subplots(len(hotel_list), 1, figsize=(12, 6 * len(hotel_list)), sharex=True)

# If there's only one hotel, `axes` won't be an iterable, so wrap it in a list
if len(hotel_list) == 1:
    axes = [axes]

# Define colors for historical and forecasted data
colors = {"historical": "navy", "forecast": "firebrick"}

# Loop through each hotel and create a separate plot
for i, hotel in enumerate(hotel_list):
    # Filter data for this hotel
    hotel_forecast = comparison[comparison["hotel"] == hotel]

    # Plot historical data
    sns.lineplot(ax=axes[i], 
                 data=hotel_forecast, 
                 x="ds", y="y", label="Historical", color=colors["historical"])

    # Plot forecasted data
    sns.lineplot(ax=axes[i], 
                 data=hotel_forecast, 
                 x="ds", y="yhat", label="Forecast", color=colors["forecast"], linestyle="dashed")

    # Formatting for each subplot
    axes[i].set_title(f"Hotel {hotel} Forecast")
    axes[i].set_xlabel("Date")
    axes[i].set_ylabel("Prices [Local Currency]")
    axes[i].legend()
    axes[i].grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show plots
plt.show()